# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('./Data/Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
!pip install langchain_community


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Hano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
#Replace None by your own value and justify
llm = ChatOpenAI(
    temperature=0.3, 
    openai_api_key)

In [18]:
prompt = ChatPromptTemplate.from_template( "Describe the product: {product_name}"
 
)

In [19]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\Hano\AppData\Local\Temp\ipykernel_28472\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [21]:
product = "Wireless Noise-Cancelling Headphones"
chain.run(product)

C:\Users\Hano\AppData\Local\Temp\ipykernel_28472\2295547843.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Wireless noise-cancelling headphones are a type of headphones that use technology to reduce or eliminate outside noise, allowing the user to enjoy their music or audio content without distractions. These headphones connect to devices such as smartphones or tablets via Bluetooth, eliminating the need for cords or cables. The noise-cancelling feature works by using built-in microphones to pick up ambient noise and then producing sound waves that cancel out those noises, creating a more immersive and peaceful listening experience. These headphones are often equipped with high-quality audio drivers for clear and crisp sound, as well as comfortable padding and adjustable headbands for long-term wear.'

## SimpleSequentialChain

In [22]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9 , openai_api_key)
# prompt template 1
first_prompt = ChatPromptTemplate.from_template("Describe the product: {product_name}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [29]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
   "Create a catchy social media ad based on this product description:\n\n{product_description}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [30]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [31]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Wireless Noise-Cancelling Headphones are headphones that utilize advanced technology to block out external noise and provide users with a high-quality audio experience. They are designed to be used without the need for a physical connection to a device, allowing for freedom of movement. These headphones are equipped with noise-cancelling capabilities, which actively reduce unwanted ambient sounds so that the user can focus on their music or audio without distractions. Additionally, they typically feature Bluetooth connectivity, long battery life, and comfortable, over-ear design for extended use. Overall, these headphones offer a premium listening experience with the added convenience of wireless technology and noise-cancelling features.
"Block out the noise and elevate your music experience with our Wireless Noise-Cancelling Headphones! 🎧 Say goodbye to distractions and hello to crystal clear sound quality. Upgrade your listening game to

'"Block out the noise and elevate your music experience with our Wireless Noise-Cancelling Headphones! 🎧 Say goodbye to distractions and hello to crystal clear sound quality. Upgrade your listening game today! #WirelessTech #NoiseCancelling #PremiumSound"'

**Repeat the above twice for different products**

## SequentialChain

In [32]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9 , openai_api_key)

first_prompt = ChatPromptTemplate.from_template(
 "Translate the following review into English:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="english_review" 
                    )


In [63]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in a short and clear sentence:\n\n{review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [70]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template("Translate the following review  :\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_review"  
                      )


In [71]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template ("Based on the translated review: {review}")
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [72]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ["review"],
    output_variables=["translated_review", "summary", "english_review", "followup_message"],
    verbose=True
)

In [73]:
review = df.Review[5]
overall_chain(review)

C:\Users\Hano\AppData\Local\Temp\ipykernel_28472\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translated_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'Le goût est mauvais, la mousse ne tient pas, peut-être un vieux lot ou une contrefaçon.',
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'followup_message': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"}

**Repeat the above twice for different products or reviews**

## Router Chain

In [48]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [49]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [50]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [53]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [54]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [55]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [56]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [57]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Hano\AppData\Local\Temp\ipykernel_28472\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [58]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [59]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [60]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures the proper functioning of the organism as a whole.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**